In [5]:
from citymobil_python_clickhouse_wrapper import ClickHouseWrapper
import pandas as pd
import numpy as np
import os
import sys
import logging
import matplotlib.pyplot as plt
import pyexasol
import datetime as dt
import h3
from sklearn.cluster import DBSCAN
from shapely.geometry import MultiPoint
from geopy.distance import great_circle
import folium

%matplotlib inline

In [6]:
cred = pd.read_json(r'/Users/skostuchik/crd_exa.json')
user = cred.login.squeeze()
password_exa = cred.password_exa.squeeze()
password_ch = cred.password_ch.squeeze()

In [7]:
connect = pyexasol.connect(dsn='ex1..3.city-srv.ru:8563', 
                           user=user, 
                           password=password_exa, 
                           fetch_dict=True)

In [8]:
d2v_hex = '''
select
       UDF.HEX(h3) hex9,
       wd,
       to_char(dt_slot, 'HH24:MI:SS') time_slot,
       sum(drivers)      as drivers,
       sum(uViews)       as uViews
from SANDBOX.d2v_voronezh_temp
group by 1, 2, 3
'''

In [9]:
hex9 = pd.DataFrame(connect.execute(d2v_hex).fetchall())

In [10]:
hex9['TIME_SLOT'] = pd.to_datetime(hex9['TIME_SLOT'])
hex9['TIME_SLOT'] = hex9['TIME_SLOT'].dt.strftime('%H:%M')

hex9['D2V'] = np.divide(hex9['DRIVERS'], hex9['UVIEWS'])

In [26]:
hex9['lat'] = hex9['HEX9'].apply(lambda x: h3.h3_to_geo(x)[0])
hex9['lon'] = hex9['HEX9'].apply(lambda x: h3.h3_to_geo(x)[1])

In [28]:
hex9

,HEX9,WD,TIME_SLOT,DRIVERS,UVIEWS,D2V,coords,lat,lon
0,088119ec105fffff,2,11:00,41.814852,44.0,0.950338,"(51.70177425373278, 39.22899436554871)",51.701774,39.228994
1,088119ed807fffff,2,17:30,2.020390,7.0,0.288627,"(51.64213581183029, 39.09015585010279)",51.642136,39.090156
2,088119ec169fffff,2,22:30,78.200370,106.0,0.737739,"(51.69647147081738, 39.19635595855114)",51.696471,39.196356
3,088119eca4bfffff,4,03:00,24.320928,41.0,0.593193,"(51.66266457914734, 39.19856209149354)",51.662665,39.198562
4,088119ecb53fffff,6,16:00,18.257144,38.0,0.480451,"(51.61724156328077, 39.234445742466775)",51.617242,39.234446
...,...,...,...,...,...,...,...,...,...
103502,088119ecb17fffff,1,03:00,0.043688,4.0,0.010922,"(51.60561361964668, 39.26811923084824)",51.605614,39.268119
103503,088119ec13bfffff,4,22:00,29.785241,21.0,1.418345,"(51.71552360458155, 39.261109117266436)",51.715524,39.261109
103504,088119ed83bfffff,7,04:30,6.497689,23.0,0.282508,"(51.65131704489111, 39.11148123585269)",51.651317,39.111481
103505,088119ecae7fffff,6,03:30,4.678900,12.0,0.389908,"(51.66162277497534, 39.297524977307035)",51.661623,39.297525


In [31]:
connect.import_from_pandas(hex9, ('SANDBOX', 'd2v_voronezh_lon_lat'))

In [29]:
hex9.columns

Index(['HEX9', 'WD', 'TIME_SLOT', 'DRIVERS', 'UVIEWS', 'D2V', 'coords', 'lat',
       'lon'],
      dtype='object')

***

In [34]:
df = pd.read_csv('/Users/skostuchik/Desktop/voronezh_d2v_geo.csv')

In [42]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [47]:
df.groupby('TIME_SLOT').agg({'UVIEWS':percentile(25)}).reset_index()

,TIME_SLOT,UVIEWS
0,00:00,15.00
1,00:30,13.00
2,01:00,11.00
3,01:30,9.00
4,02:00,7.00
5,02:30,7.00
6,03:00,7.00
7,03:30,6.00
8,04:00,5.00
9,04:30,6.00


In [53]:
np.quantile(df.UVIEWS, 0.5)

35.0

In [60]:
df[(df['WD']==1)&(df['TIME_SLOT']=='07:00')]

,HEX9,WD,TIME_SLOT,DRIVERS,UVIEWS,D2V,ID,POLYGON_ID,POLYGON_NAME,PRICE,POLY
59,088119ecadbfffff,1,07:00,1.670911,19,0.087943,NaN,NaN,NaN,NaN,NaN
106,088119ed9b9fffff,1,07:00,6.695910,32,0.209247,NaN,NaN,NaN,NaN,NaN
253,088119ec155fffff,1,07:00,19.119669,81,0.236045,NaN,NaN,NaN,NaN,NaN
789,088119ecae7fffff,1,07:00,3.666686,30,0.122223,NaN,NaN,NaN,NaN,NaN
3277,088119ed995fffff,1,07:00,12.524573,57,0.219729,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
103605,088119ed8c7fffff,1,07:00,36.483175,105,0.347459,NaN,NaN,NaN,NaN,NaN
103690,088119ed997fffff,1,07:00,18.258175,70,0.260831,NaN,NaN,NaN,NaN,NaN
103697,088119ed889fffff,1,07:00,42.169169,137,0.307804,NaN,NaN,NaN,NaN,NaN
104197,088119ed8ebfffff,1,07:00,22.880268,120,0.190669,NaN,NaN,NaN,NaN,NaN


***

In [145]:
import pyproj    
import shapely
import shapely.wkt
import shapely.ops as ops
from shapely.geometry.polygon import Polygon
from shapely.geometry import MultiPolygon
from functools import partial

#import shapely.geometry

<b>Area of Polygon</b>

In [96]:
geom = Polygon([(39.169650077819824, 51.714320108267735), (39.16750431060791, 51.710171938545585),
                (39.181838035583496, 51.70620954941852), (39.18132305145264, 51.71104946771399),
                (39.176387786865234, 51.71187379778839), (39.180378913879395, 51.7072201245831),
                (39.169349670410156, 51.71070377643754), (39.169735908508294, 51.7131235598911),
                (39.17471408843994, 51.71229925259395), (39.169650077819824, 51.714320108267735)])

geom_area = ops.transform(
    partial(
        pyproj.transform,
        pyproj.Proj(init='EPSG:4326'),
        pyproj.Proj(
            proj='aea',
            lat_1=geom.bounds[1],
            lat_2=geom.bounds[3]
        )
    ),
    geom)

/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [97]:
# Print the area in m^2
print(geom_area.area)

231804.8930212317


<b>Area of Multipolygon</b>

In [131]:
coords_list = [[(39.169650077819824, 51.714320108267735), (39.16750431060791, 51.710171938545585),
               (39.181838035583496, 51.70620954941852), (39.18132305145264, 51.71104946771399),
               (39.176387786865234, 51.71187379778839), (39.180378913879395, 51.7072201245831),
               (39.169349670410156, 51.71070377643754), (39.169735908508294, 51.7131235598911),
               (39.17471408843994, 51.71229925259395), (39.169650077819824, 51.714320108267735)],
              [(39.17741775512695, 51.716952403267), (39.177889823913574, 51.7150114329008),
               (39.18231010437012, 51.71570274696624), (39.17741775512695, 51.716952403267)]]

In [137]:
list_polygons =  [Polygon(i) for i in coords_list]

In [146]:
geom_area = ops.transform(
    partial(
        pyproj.transform,
        pyproj.Proj(init='EPSG:4326'),
        pyproj.Proj(
            proj='aea',
            lat_1=geom.bounds[1],
            lat_2=geom.bounds[3]
        )
    ),
    MultiPolygon(list_polygons))

/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/skostuchik/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:306: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))


In [147]:
# Print the area in m^2
print(geom_area.area)

266046.56593898695
